In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions

spark = SparkSession.builder\
    .appName("spark_fpk_test")\
    .getOrCreate() 


stg_acc = spark.read.format("jdbc")\
    .option("url", "jdbc:sqlserver://fpc-crm-dbt.fpc.org.rzd:1433;encrypt=true;trustServerCertificate=true;databaseName=DWHDB") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .option("useUnicode", "true") \
    .option("continueBatchOnError", "true") \
    .option("fetchSize", "10000") \
    .option("user", "sasuser") \
    .option("password", "Orion123") \
    .option("dbtable", "(select top(10) id from etl_stg.STG_ACCOUNT) as tbl") \
    .load()
#Формируем df source
stg_acc = spark.read.format("jdbc")\
    .option("url", "jdbc:sqlserver://fpc-crm-dbt.fpc.org.rzd:1433;encrypt=true;trustServerCertificate=true;databaseName=DWHDB") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .option("useUnicode", "true") \
    .option("continueBatchOnError", "true") \
    .option("fetchSize", "10000") \
    .option("user", "sasuser") \
    .option("password", "Orion123") \
    .option("dbtable", "etl_stg.STG_ACCOUNT") \
    .load()
stg=stg_acc.drop('ORGANISATION_ID')
#Получаем последний идентификатор
max = spark.read.format("jdbc")\
    .option("url", "jdbc:sqlserver://fpc-crm-dbt.fpc.org.rzd:1433;encrypt=true;trustServerCertificate=true;databaseName=DWHDB") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .option("useUnicode", "true") \
    .option("continueBatchOnError", "true") \
    .option("fetchSize", "10000") \
    .option("user", "sasuser") \
    .option("password", "Orion123") \
    .option("dbtable", "(select case when MAX_SK is null then 0 else MAX_SK end  as MAX_SK from etl_int.MAX_DWH_SK where TABLE_NAME='DIM_ACCOUNT') as Tbl")\
    .load()
#Формируем df target
dwh = spark.read.format("jdbc")\
    .option("url", "jdbc:sqlserver://fpc-crm-dbt.fpc.org.rzd:1433;encrypt=true;trustServerCertificate=true;databaseName=DWHDB") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .option("useUnicode", "true") \
    .option("continueBatchOnError", "true") \
    .option("fetchSize", "10000") \
    .option("user", "sasuser") \
    .option("password", "Orion123") \
    .option("dbtable", "dwh.DIM_ACCOUNT")\
    .load()
#Формируем темповые вью для sparkSQL
stg.createOrReplaceTempView("stg")
max.createOrReplaceTempView("max")
dwh.createOrReplaceTempView("dwh")
#Формируем temp (join source&target)
tmp=spark.sql("select case when DWH.ACCOUNT_SK is null then (select MAX_SK from MAX) + row_number() over(order by DWH.ACCOUNT_SK)else DWH.ACCOUNT_SK end as ACCOUNT_DIM_SK,STG.ID,STG.ACCOUNT_NUMBER,STG.TYPE,STG.BONUS,STG.QUALIFICATION,STG.CURRENT_LEVEL,STG.STATUS,DATE(STG.CREATED_DATE) as CREATED_DATE,DATE(STG.CLOSED_DATE) as CLOSED_DATE,STG.SWINDLER,STG.IS_LEVEL_CONFIRMED,DATE(STG.NEGATIVE_BALANCE_DATE) as NEGATIVE_BALANCE_DATE,STG.BONUS_FPK,STG.BONUS_PARTNER,STG.MERGED_ACTIVE_ACCOUNT_ID from stg left join dwh on stg.ACCOUNT_NUMBER = dwh.ACCOUNT_BK1")
#Пишем temp в БД
tmp.write.format('jdbc')\
    .option("url", "jdbc:sqlserver://fpc-crm-dbt.fpc.org.rzd:1433;encrypt=true;trustServerCertificate=true;databaseName=DWHDB") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .option("useUnicode", "true") \
    .option("continueBatchOnError", "true") \
    .option("user", "sasuser") \
    .option("password", "Orion123") \
    .option("dbtable", "etl_int.tmp_ACCOUNT") \
    .mode('append').save()